In [1]:
import numpy as np
import matplotlib.pyplot as plt
from read_co2 import read_co2
from chi_sq_fit_poly import chi_square_fit_poly
import math
from scipy.optimize import fmin_bfgs

## Previous estimate using the $\chi^2$ polynomial fit

In [2]:
x,y = read_co2('co2_mm_mlo.txt')

n = len(y)   # number of data points

err = np.ones(n)

[af0, af1, af2, chi_sq] = chi_square_fit_poly(x,y,err)

# Print out results
print (' Least squares fit of', n, 'data points')
print (' -----------------------------------')
print ("    a2 = {0:6.4f} ".format( af2))
print ("    a1 = {0:6.4f} ".format( af1))
print ("    a0 = {0:3.4f} ".format( af0))
print (" chi-square value : ", chi_sq/(n-3) )


Some random stuff!!!
 Least squares fit of 720 data points
 -----------------------------------
    a2 = 0.0126 
    a1 = -48.4672 
    a0 = 46997.2984 
 chi-square value :  5.040122311008971


## $\chi^2$ function and derivatives  

In [3]:
def f(A):
    a0 = A[0]
    a1 = A[1]
    a2 = A[2]
    return np.sum(((y - a0 - a1*x - a2*x**2) / err)**2)

def df(A) :
    a0 = A[0]
    a1 = A[1]
    a2 = A[2]    
    df0 = -2*np.sum( ((y - a0 - a1*x - a2*x**2) / err**2) )
    df1 = -2*np.sum( ((y - a0 - a1*x - a2*x**2)*x / err**2) )
    df2 = -2*np.sum( ((y - a0 - a1*x - a2*x**2)*(x**2) / err**2) )       
    return np.array([df0,df1,df2])

## Run BFGS Algorithm  

In [4]:
A0 = np.array([0.01,2.0,0.001])

print(f(A0))

def make_minimize_cb(path=[]):    
    def minimize_cb(xk):
        # note that we make a deep copy of xk
        path.append(np.copy(xk))
    return minimize_cb



path = [A0]
gtol = 1e-6
res = fmin_bfgs(f=f, fprime=df, x0=A0, gtol=gtol, callback=make_minimize_cb(path))
print (res)
path = np.array(path).T
print(path)
print(path.shape)

41333016362.16382
         Current function value: 3506.795746
         Iterations: 8
         Function evaluations: 146
         Gradient evaluations: 134
[ 4.69947620e+04 -4.84646545e+01  1.25759698e-02]
[[ 1.00000000e-02  9.99999952e-03  6.79356200e-03  6.81272356e+03
   2.17410332e+04  4.69947620e+04  4.69947620e+04  4.69947620e+04
   4.69947620e+04]
 [ 2.00000000e+00  1.99999904e+00 -1.18803953e+00 -8.04161685e+00
  -2.30594617e+01 -4.84646545e+01 -4.84646545e+01 -4.84646545e+01
  -4.84646545e+01]
 [ 1.00000000e-03 -9.16311777e-04  6.86830785e-04  2.41037072e-03
   6.18706339e-03  1.25759698e-02  1.25759698e-02  1.25759698e-02
   1.25759698e-02]]
(3, 9)


## Print final Output as above

In [5]:
# Print out results
print (' BFGS evaluation of fit parameters for', n, 'data points')
print (' -----------------------------------')
print ("    a2 = {0:6.4f} ".format( res[2]))
print ("    a1 = {0:6.4f} ".format( res[1]))
print ("    a0 = {0:3.4f} ".format( res[0]))
print (" chi-square value : ", f(res)/(n-3) )


 BFGS evaluation of fit parameters for 720 data points
 -----------------------------------
    a2 = 0.0126 
    a1 = -48.4647 
    a0 = 46994.7620 
 chi-square value :  4.890928515869838
